#데이터 로드 및 전처리
3축 가속도계(accelerometer)와 3축 자이로스코프(gyroscope)를 활용해 측정된 센서 데이터

- train_features.csv (1875000, 8)

> id 별 600 time 간 동작 데이터

>id 3125개 x 600 time =1875000 데이터


- train_labels.csv (3125, 3)

>id 별 동작과 동작 label(61개)

- test_features.csv (469200, 8)

>id 별 600 time간 동작 데이터

>id 782개 x 600 time =469200 데이터

-sample_submission.csv (782, 62)

>id별 동작을 예측해 작성하는 csv

In [3]:
import math
import time
from itertools import chain
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
from tqdm.auto import tqdm
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR

In [42]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train_features.csv")
train_labels = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train_labels.csv")
test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_features.csv')
submission=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sample_submission.csv')
train

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234
...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437
1874996,3124,596,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087
1874997,3124,597,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125
1874998,3124,598,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478


In [43]:
#id 별로 각각 600개의 데이터가 있음을 확인   
train['id'].value_counts()

2047    600
1347    600
2378    600
1354    600
2379    600
       ... 
1699    600
675     600
2722    600
1698    600
0       600
Name: id, Length: 3125, dtype: int64

In [45]:
mean=train.iloc[:,2:].mean(axis=0)
std=train.iloc[:,2:].std(axis=0)

X_train_lstm=np.zeros((3125,600,6))
for uid in range(3125):
    df=train[train.id==uid].iloc[:,2:]
    X_train_lstm[uid,:,:]=(df-mean)/std
    
X_test_lstm=np.zeros((782,600,6))
for uid in range(782):
    df=test[test.id==uid+3125].iloc[:,2:]
    X_test_lstm[uid,:,:]=(df-mean)/std

In [46]:
X_train_lstm.shape

(3125, 600, 6)

In [50]:
y = tf.keras.utils.to_categorical(train_labels['label']) 
y.shape

(3125, 61)

In [54]:
non_ex_idx=train_labels[train_labels['label_desc']=='Non-Exercise'].index
ex_idx=train_labels[train_labels['label_desc']!='Non-Exercise'].index

X_tr_ex=X_train_lstm[ex_idx,:,:]
y_tr_ex=y[ex_idx,:]

X_tr_non_ex=X_train_lstm[non_ex_idx,:,:]
y_tr_non_ex=y[non_ex_idx,:]

In [55]:
def aug(data,shift=0):
    shift_data=np.roll(data,shift,axis=1)
    return shift_data

X_train_aug=np.zeros((1607*12,600,6))
for i in range(12):
    aug_data=aug(X_tr_ex,int(np.random.random()*600))
    X_train_aug[1607*i:1607*(i+1),:,:]=aug_data
    
y_train_aug=np.vstack([y_tr_ex]*12)

In [56]:
X_tr_aug=np.vstack([X_tr_non_ex,X_train_aug])
y_tr_aug=np.vstack([y_tr_non_ex,y_train_aug])

In [62]:
X_tr_aug.shape

(20802, 600, 6)

In [74]:
train.columns[2:8]

Index(['acc_x', 'acc_y', 'acc_z', 'gy_x', 'gy_y', 'gy_z'], dtype='object')

In [101]:
#0~599개
def sliding_window(array):
   time_dictionary_window=[]
   window = []
   start_point = 0
   end_point = 600
   for i in tqdm(array):
      for cursor in range(start_point, end_point - 6, 3):
        my_df = pd.DataFrame(data= i, columns= train.columns[2:8])
        data = np.array(my_df.iloc[cursor : cursor+6])
        time_dictionary_window.append(data)
        #window.append(data)

   return time_dictionary_window

In [83]:
time_window = sliding_window(X_tr_aug)
time_array = np.array(time_window)
time_array.shape

(4118796, 6, 6)

In [106]:
X_test_lstm.shape

(782, 600, 6)

In [107]:
time_window_test = sliding_window(X_test_lstm)
time_array_test = np.array(time_window_test)
time_array_test.shape

(154836, 6, 6)

In [84]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [86]:
time_array=time_array.astype(float)
X=tf.reshape(time_array,[-1, 198, 36])
X.shape

TensorShape([20802, 198, 36])

In [92]:
y_tr_aug.shape

(20802, 61)

In [91]:
model = keras.Sequential([
keras.layers.Conv1D(filters=16,kernel_size=7,strides=1,padding='same',
                    activation='relu',input_shape=(198,36)),
keras.layers.BatchNormalization(),
keras.layers.MaxPooling1D(pool_size=2,strides=2),
keras.layers.Conv1D(filters=32,kernel_size=5,strides=1,
                                 padding='same',activation='relu'),
keras.layers.BatchNormalization(),
keras.layers.MaxPooling1D(pool_size=2,strides=2),
keras.layers.Conv1D(filters=64,kernel_size=3,strides=1,
                    padding='same',activation='relu'),
keras.layers.BatchNormalization(),
keras.layers.MaxPooling1D(pool_size=2,strides=2),
keras.layers.LSTM(180,return_sequences=True),
keras.layers.LSTM(180),
keras.layers.BatchNormalization(),
keras.layers.Dense(180,activation='relu'),
keras.layers.BatchNormalization(),
keras.layers.Dense(61,activation='softmax')])

model.compile(loss='categorical_crossentropy',
                     optimizer='rmsprop',
                     metrics =['categorical_crossentropy'])
#model.summary()

In [93]:
check=keras.callbacks.ModelCheckpoint('conv1d_lstm_1.h5',save_best_only=True)
earlystop=keras.callbacks.EarlyStopping(patience=5,restore_best_weights=True)
history=model.fit(X,y_tr_aug,epochs=10,
                    validation_split=0.2,verbose=1,batch_size=32,
                    callbacks=[check,earlystop])

Epoch 1/10
521/521 [==============================] - 81s 146ms/step - loss: 2.1694 - categorical_crossentropy: 2.1694 - val_loss: 0.9074 - val_categorical_crossentropy: 0.9074
Epoch 2/10
521/521 [==============================] - 74s 142ms/step - loss: 0.9584 - categorical_crossentropy: 0.9584 - val_loss: 0.6235 - val_categorical_crossentropy: 0.6235
Epoch 3/10
521/521 [==============================] - 75s 144ms/step - loss: 0.6695 - categorical_crossentropy: 0.6695 - val_loss: 0.5129 - val_categorical_crossentropy: 0.5129
Epoch 4/10
521/521 [==============================] - 81s 155ms/step - loss: 0.5324 - categorical_crossentropy: 0.5324 - val_loss: 0.3584 - val_categorical_crossentropy: 0.3584
Epoch 5/10
521/521 [==============================] - 75s 144ms/step - loss: 0.4249 - categorical_crossentropy: 0.4249 - val_loss: 0.3605 - val_categorical_crossentropy: 0.3605
Epoch 6/10
521/521 [==============================] - 74s 142ms/step - loss: 0.3419 - categorical_crossentropy: 0.3

In [108]:
test_X=tf.reshape(time_array_test,[-1, 198,  36])
test_X.shape

TensorShape([782, 198, 36])

In [109]:
prediction=model.predict(test_X)

In [110]:
prediction.shape

(782, 61)

In [111]:
submission

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,3125,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3126,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3127,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3128,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3129,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
778,3903,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
779,3904,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
780,3905,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [112]:
submission.iloc[:,1:]=prediction

In [113]:
submission

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,3125,9.669675e-05,4.723570e-07,2.099399e-07,5.654878e-07,4.019695e-07,9.206414e-07,5.174873e-06,1.196940e-08,1.555431e-07,7.074202e-05,2.264483e-06,4.329570e-02,3.080977e-06,9.564142e-01,2.468282e-06,1.420245e-07,1.146632e-06,2.508417e-08,9.484902e-08,7.301361e-08,1.054495e-07,4.451078e-07,3.825658e-07,2.787430e-07,3.631755e-05,1.371488e-07,7.295873e-07,3.931366e-06,7.670439e-07,3.715587e-07,5.162730e-07,4.854151e-07,2.833700e-07,4.969535e-07,7.436895e-06,4.925278e-07,3.278460e-06,1.413137e-06,1.495936e-07,1.091443e-06,1.344835e-06,2.984926e-07,2.540357e-07,1.006321e-07,4.898923e-07,8.550136e-08,5.794467e-08,2.768129e-07,3.748718e-08,1.109217e-06,2.185573e-08,3.617400e-05,1.259909e-06,6.702454e-09,2.247208e-06,5.122641e-07,9.252717e-08,9.065847e-08,4.898400e-07,4.060085e-08,1.458432e-06
1,3126,8.319114e-07,1.464545e-06,1.641234e-06,3.935331e-07,1.172704e-05,2.413450e-08,3.376026e-07,3.821468e-08,8.723234e-08,7.275789e-07,2.114510e-07,7.828564e-07,1.537673e-07,1.072583e-06,6.426898e-08,6.113530e-08,4.683340e-08,1.202235e-09,1.861245e-08,1.820037e-09,1.183128e-07,2.059534e-07,1.337948e-08,2.200816e-06,1.742473e-07,6.756432e-08,9.999442e-01,3.738273e-08,1.004343e-06,1.132380e-06,3.369370e-06,3.705081e-08,3.657303e-07,2.156308e-06,9.949026e-07,7.383521e-10,9.899022e-08,8.803718e-08,1.098096e-07,2.798572e-06,2.068271e-07,2.183996e-07,3.772159e-06,5.325965e-07,2.000965e-08,4.486883e-06,1.260239e-08,2.966437e-06,1.975924e-06,2.843972e-08,5.526180e-08,3.853213e-07,2.763919e-06,2.759508e-07,7.604889e-07,1.046195e-07,5.960132e-08,1.553112e-07,8.869314e-08,2.672851e-07,1.968298e-06
2,3127,1.214731e-09,1.827638e-07,3.427459e-07,2.496057e-10,3.129405e-08,6.485694e-08,1.137345e-07,7.114570e-09,2.281196e-08,1.012882e-09,3.095579e-08,1.410550e-08,2.205569e-08,1.507022e-07,7.964557e-08,5.833430e-09,6.098048e-08,5.643933e-09,2.085882e-08,6.928298e-08,2.740084e-06,1.400919e-06,2.528929e-08,2.740810e-08,1.743950e-08,4.952961e-08,1.168241e-07,2.373423e-07,2.307621e-07,1.589115e-08,1.966682e-09,2.576001e-08,1.701927e-09,1.000726e-07,1.400715e-07,7.286577e-10,5.106268e-08,9.553217e-09,5.912375e-10,1.411004e-07,1.451858e-07,1.024151e-07,1.358022e-08,1.834839e-07,1.990520e-06,9.999797e-01,4.674548e-07,7.362816e-06,1.974656e-07,1.238555e-09,1.807630e-08,1.418641e-09,1.474180e-08,6.423316e-08,2.200000e-06,3.107197e-08,5.668258e-10,5.201861e-07,3.173048e-07,3.063070e-08,9.337352e-08
3,3128,4.856175e-05,1.462057e-06,3.276534e-05,1.038737e-06,2.318050e-05,5.851986e-07,3.234459e-05,1.727797e-06,2.376329e-05,3.553883e-07,8.040928e-07,8.176859e-07,2.310036e-06,8.073715e-07,8.697019e-07,1.760502e-07,1.588435e-06,5.770470e-07,1.281328e-04,7.375238e-07,3.099808e-07,9.065065e-07,1.808238e-07,3.076319e-05,2.293792e-05,6.094881e-06,9.991925e-01,1.645037e-05,4.305070e-05,1.234411e-05,1.594632e-06,6.418387e-06,2.309541e-07,2.455299e-06,1.134141e-06,3.356604e-06,8.817136e-07,1.266421e-06,1.417690e-05,1.522417e-05,1.544080e-06,4.012432e-06,1.099103e-06,7.463015e-07,3.817645e-06,6.041653e-07,4.608317e-07,4.837452e-06,5.160773e-06,1.198679e-05,2.360110e-04,2.921278e-06,1.660823e-06,5.215639e-06,1.315206e-06,5.837274e-06,9.872119e-06,1.765576e-07,5.113638e-06,2.820642e-07,2.231955e-05
4,3129,3.620257e-03,2.973065e-04,1.594244e-03,3.260569e-04,3.938262e-05,1.022165e-04,1.685095e-04,4.140988e-05,8.097255e-04,2.938387e-04,1.092352e-04,1.125940e-03,2.358426e-04,3.748558e-03,9.636830e-03,1.022868e-04,4.717346e-05,9.628305e-05,5.858287e-05,2.246543e-05,6.821896e-06,7.174048e-05,6.093133e-03,7.543489e-05,5.443729e-04,5.639135e-06,8.816046e-01,3.398373e-05,1.312292e-03,3.709921e-04,1.686236e-03,7.226413e-05,1.846814e-03,7.978416e-04,6.661151e-05,7.729377e-05,4.119336e-05,3.736979e-04,5.892503e-04,6.474352e-04,1.940689e-04,2.548926e-04,4.783507e-02,2.570223e-03,5.915932e-04,1.490559e-04,3.634774e-05,5.853

In [114]:
submission.to_csv('submission.csv', index=False)